# LSTM. 8 featerow - mag, x, y, depth, dist, distance, plate, days_diff

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
SEED = 1337
tf.random.set_seed(SEED)
np.random.seed(SEED)

2024-01-20 19:07:31.428825: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-20 19:07:31.429275: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-20 19:07:31.430933: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-20 19:07:31.679681: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-20 19:07:34.361954: W tensorflow/compiler/

In [2]:
ds_train = tf.data.Dataset.load("../data/ds_train_7/")
ds_val = tf.data.Dataset.load("../data/ds_val_7/")
ds_test = tf.data.Dataset.load("../data/ds_test_7/")

2024-01-20 19:07:39.399796: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-20 19:07:40.016371: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-20 19:07:40.016457: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-20 19:07:40.026201: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-20 19:07:40.026685: I tensorflow/compile

In [3]:
ds_train.cardinality().numpy(), ds_val.cardinality().numpy(), ds_test.cardinality().numpy()

(3412357, 485378, 105236)

In [4]:
BATCH_SIZE = 1024
BLOCK_SIZE = 32

In [5]:
def fix_x_and_y(features, context, labels):
    return features, labels

In [6]:
ds_train = ds_train.map(fix_x_and_y)
ds_val = ds_val.map(fix_x_and_y)
ds_test = ds_test.map(fix_x_and_y)

In [7]:
for x, y in ds_train.batch(BATCH_SIZE).take(1):
    print(x.shape)
    print(x[:,:,:-3].shape, x[:,:,-3].shape, x[:,:,-2].shape, x[:,:,-1].shape)
    print(y.shape)

(1024, 32, 9)
(1024, 32, 6) (1024, 32) (1024, 32) (1024, 32)
(1024, 1)


In [8]:
ys = []
for x, y in ds_train.batch(BATCH_SIZE):
    ys.append(y.numpy())
ys = np.concatenate(ys)
np.bincount(tf.cast(ys, tf.int32).numpy().flatten())

array([2685799,  726558])

In [9]:
#neg, pos = np.bincount(tf.cast(ys, tf.int32).numpy().flatten())
neg, pos = 2685799, 726558
total = neg + pos
total, pos, neg

(3412357, 726558, 2685799)

In [10]:
ds_train = ds_train.batch(BATCH_SIZE).shuffle(2000, seed=SEED).prefetch(tf.data.experimental.AUTOTUNE)
ds_val = ds_val.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [11]:
@tf.keras.saving.register_keras_serializable()
class MyModel(tf.keras.Model):
    def __init__(self, n_embed):
        super(MyModel, self).__init__()
        self.embed_plate = tf.keras.layers.Embedding(32, n_embed // 9, name="embed_plate")
        self.embed_dd = tf.keras.layers.Embedding(20, n_embed // 9, name="embed_dd")
        self.embed_magtype = tf.keras.layers.Embedding(20, n_embed // 9, name="embed_magtype")
        self.embed = tf.keras.layers.Dense(n_embed // 9 * 6, name="embed_continous")
        self.lstm1 = tf.keras.layers.LSTM(n_embed, return_sequences=True, name="lstm1")
        self.lstm2 = tf.keras.layers.LSTM(n_embed, name="lstm2")
        self.dense1 = tf.keras.layers.Dense(n_embed, activation='relu', name="pre_classifier")
        self.dense2 = tf.keras.layers.Dense(1, activation='sigmoid', name="classifier")
        self.conc = tf.keras.layers.Concatenate(axis=-1, name="concatenate")

    def build_graph(self):
        x = [tf.keras.layers.Input(shape=(BLOCK_SIZE, 6), batch_size=None, name="input_continous"),
            tf.keras.layers.Input(shape=(BLOCK_SIZE), batch_size=None, name="input_plate"),
            tf.keras.layers.Input(shape=(BLOCK_SIZE), batch_size=None, name="input_dd"),
            tf.keras.layers.Input(shape=(BLOCK_SIZE), batch_size=None, name="input_magtype")]
        return tf.keras.Model(inputs=x, outputs=self.call(x))

    def call(self, x):
        cont, plate, dd, magtype = x[:,:,:-3], x[:,:,-3], x[:,:,-2], x[:,:,-1]
        x1 = self.embed(cont)
        x2 = self.embed_plate(plate)
        x3 = self.embed_dd(dd)
        x4 = self.embed_magtype(magtype)
        x = self.conc([x1, x2, x3, x4])
        x = self.lstm1(x)
        x = self.lstm2(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return x

In [12]:
model = MyModel(135)

In [13]:
model.build(input_shape=(None, BLOCK_SIZE, 9))
model.count_params()

312886

In [14]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
            optimizer='adam',
            metrics=['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

In [15]:
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)
class_weight = {0: weight_for_0, 1: weight_for_1}
history = model.fit(ds_train, epochs=20, validation_data=ds_val, class_weight=class_weight)

Epoch 1/20


2024-01-20 19:15:30.114093: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 204 of 2000
2024-01-20 19:15:50.102839: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 610 of 2000
2024-01-20 19:16:00.108645: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 811 of 2000
2024-01-20 19:16:10.117405: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 943 of 2000
2024-01-20 19:16:30.114923: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 1303 of 2000
2024-01-20 19:16:40.117795: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a

3333/3333 [==============================] - 645s 154ms/step - loss: 0.5532 - accuracy: 0.7518 - recall: 0.6235 - precision: 0.4413 - val_loss: 0.5735 - val_accuracy: 0.7085 - val_recall: 0.4510 - val_precision: 0.4248
Epoch 2/20


2024-01-20 19:26:10.002510: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 136 of 2000
2024-01-20 19:26:29.949957: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 406 of 2000
2024-01-20 19:26:49.954741: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 679 of 2000
2024-01-20 19:27:00.030836: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 818 of 2000
2024-01-20 19:27:20.109773: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 1076 of 2000
2024-01-20 19:27:39.979307: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a

3333/3333 [==============================] - 669s 157ms/step - loss: 0.5195 - accuracy: 0.7658 - recall: 0.6759 - precision: 0.4656 - val_loss: 0.5876 - val_accuracy: 0.6937 - val_recall: 0.4462 - val_precision: 0.4016
Epoch 3/20


...

3333/3333 [==============================] - 410s 88ms/step - loss: 0.2298 - accuracy: 0.8796 - recall: 0.9237 - precision: 0.6539 - val_loss: 1.6313 - val_accuracy: 0.6925 - val_recall: 0.4507 - val_precision: 0.4008
Epoch 19/20


2024-01-20 22:00:24.893591: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 183 of 2000
2024-01-20 22:00:34.906180: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 365 of 2000
2024-01-20 22:00:44.923821: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 547 of 2000
2024-01-20 22:01:05.084617: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 926 of 2000
2024-01-20 22:01:24.890618: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 1290 of 2000
2024-01-20 22:01:34.925251: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a

3333/3333 [==============================] - 364s 77ms/step - loss: 0.2161 - accuracy: 0.8868 - recall: 0.9310 - precision: 0.6681 - val_loss: 1.5497 - val_accuracy: 0.6818 - val_recall: 0.4496 - val_precision: 0.3859
Epoch 20/20


2024-01-20 22:06:29.217410: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 189 of 2000
2024-01-20 22:06:49.203244: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 612 of 2000
2024-01-20 22:07:09.173472: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 1050 of 2000
2024-01-20 22:07:19.202104: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 1267 of 2000
2024-01-20 22:07:39.163900: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 1688 of 2000
2024-01-20 22:07:49.176865: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take

3333/3333 [==============================] - 331s 71ms/step - loss: 0.2046 - accuracy: 0.8929 - recall: 0.9361 - precision: 0.6806 - val_loss: 1.6493 - val_accuracy: 0.6889 - val_recall: 0.4596 - val_precision: 0.3973


In [16]:
model.evaluate(ds_test)

103/103 [==============================] - 8s 76ms/step - loss: 1.0684 - accuracy: 0.7449 - recall: 0.6481 - precision: 0.3999


[1.0684369802474976,
 0.7448877096176147,
 0.6481326222419739,
 0.39988434314727783]

In [17]:
model.save("../models/model_v9_cw.keras")

In [18]:
# save history
history = pd.DataFrame.from_dict(history.history)
history.to_csv("../models/history_model_v9_cw.csv", index=False)